# Termpaper TECH2

## NHH Fall 2024 - Group 9: Alexander Solheim, Aurora Malthe-Sørenssen, Balázs Biró
### Candidate numbers: 7, xx, xx
---

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### P.1 Preliminary data handling and cleaning
Since the data is in multiple csv files - concatenate all into a single dataframe and check for data consistency


In [ ]:
# Specify variables for data handling
data_path = 'data/'
regions = ["NO2", "DE_LU"]
datasets = ["DayAheadPrices_12.1.D", "PhysicalFlows_12.1.G"]
csv_sep = "	"
start_year, end_year = 2019, 2023

def read_csvs(dataset, start_year, end_year, csv_sep):
    """ Read multiple csv files into a single pandas dataframe """
    data = pd.DataFrame()
    for year in range(start_year, end_year+1):
        for month in range(1, 13):
            file_path = f"{data_path}/{dataset}/{year}_{month:02d}_{dataset}.csv"
            try:
                df = pd.read_csv(file_path, sep=csv_sep)
                data = pd.concat([data, df], ignore_index=True)
            except FileNotFoundError:
                pass
    return data


#### P.1.1 - DayAheadPrices
- Keep relevant columns
- Keep only the two selected regions
- Reset indeces for clarity

In [ ]:
# Create the dataframe for DayAheadPrices and check for consistency
dayaheadprices = pd.DataFrame()
keep_columns = ["DateTime", "ResolutionCode", "MapCode", "Price"] # Relevant columns for the analysis
# Keep only the selected MapCodes and relevant columns
dayaheadprices = pd.concat([dayaheadprices, read_csvs(datasets[0], start_year, end_year, csv_sep)])[keep_columns]
dayaheadprices = dayaheadprices.loc[dayaheadprices["MapCode"].isin(regions)]

# Reset indeces
dayaheadprices.reset_index(drop=True, inplace=True)
    
# Check for duplicates and missing values
print(f"Duplicate values: {dayaheadprices.duplicated().sum()}")
print(f"Missing values: {dayaheadprices.isnull().sum().sum()}")


#### P.1.2 - PhysicalFlows
- Keep only the trades between NO2 and DE_LU and vice versa


In [ ]:
physicalflows = pd.DataFrame()
# Keep only if InMapCode or OutMapCode is NO2 or DE_LU

physicalflows = pd.concat([physicalflows, read_csvs(datasets[1], start_year, end_year, csv_sep)])

# Keep only trades between the selected regions
physicalflows = physicalflows.loc[(physicalflows["InMapCode"].isin(regions)) & (physicalflows["OutMapCode"].isin(regions))]
physicalflows.reset_index(drop=True, inplace=True)

# Check for duplicates and missing values
print(f"Duplicate values: {physicalflows.duplicated().sum()}")
print(f"Missing values: {physicalflows.isnull().sum().sum()}")


## Task 1
### The development of energy prices across the sample period
- Visualisation of energy prices in NO2 and DE_LU per hour using a line plot
- Showing the official opening time of Nordlink in Dec. 9. 2020

In [ ]:
# Create a copy from dayaheadprices and filtering to hourly data only
dayaheadprices_hourly = dayaheadprices.loc[dayaheadprices["ResolutionCode"] == "PT60M"].copy()
# Convert DateTime to datetime object
dayaheadprices_hourly["DateTime"] = pd.to_datetime(dayaheadprices_hourly["DateTime"])

# Plot the hourly prices for the selected regions on two subplots
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
for region, ax in zip(regions, axes):
    sns.lineplot(data=dayaheadprices_hourly.loc[dayaheadprices_hourly["MapCode"] == region], x="DateTime", y="Price", ax=ax)
    ax.axvline(pd.Timestamp("2020-12-09"), color='red', linestyle='--', label='Nordlink Opening')
    ax.set_title(f"Day-ahead prices for {region}")
    ax.set_ylabel("Price [€/MWh]")
    ax.set_xlabel("Date")
plt.tight_layout()
# Save to file
plt.savefig("figure_task1.png")

Summary

## Task 2
### Statistics table for the day ahead price data
- Contains the mean, median, standard deviation, min and max for hourly price for each year rounded to two digits

In [ ]:
# Create a year variable
dayaheadprices_hourly["Year"] = dayaheadprices_hourly["DateTime"].dt.year

# Calculate the summary statistics for the hourly prices
summary = dayaheadprices_hourly.groupby(["Year", "MapCode"])["Price"].agg(["mean", "median", "std", "min", "max"]).round(2).reset_index()
print(summary)
# Save to excel file
summary.to_excel("table_task2.xlsx", index=False)

Summary

## Task 3
### Annual sum of exports and imports of electricity in NO2 on Nordlink
- Yearly breakdown
- Nordlink only

In [ ]:
# Transform to datetime object and create a year variable
physicalflows["DateTime"] = pd.to_datetime(physicalflows["DateTime"])
physicalflows["Year"] = physicalflows["DateTime"].dt.year

# Calculate the import and export flows
imports, exports = physicalflows[physicalflows["InMapCode"] == "NO2"].groupby("Year")["FlowValue"].sum().reset_index(), physicalflows[physicalflows["OutMapCode"] == "NO2"].groupby("Year")["FlowValue"].sum().reset_index()
imports["FlowType"], exports["FlowType"] = "Import", "Export"

flow_data = pd.concat([imports, exports])

# Plot the import and export flows in million MWh
sns.barplot(data=flow_data, x="Year", y="FlowValue", hue="FlowType", palette=["orange", "green"])
plt.title("Annual imports and exports of electricity in NO2 with Germany") 
plt.ylabel("Total electricity flow [in Million MWh]")
plt.yticks(ticks=plt.yticks()[0], labels=[f'{int(tick/1e6)}' for tick in plt.yticks()[0]])
plt.savefig("figure_task3.png")
plt.show()


In [ ]:
# Show the how much % was the export compared to the import each year
percentage, years = (exports["FlowValue"] / imports["FlowValue"] * 100).round(2), imports["Year"]
percentage_data = pd.DataFrame({"Year": years, "Percentage": percentage})
sns.lineplot(data=percentage_data, x="Year", y="Percentage", marker="o")
plt.title("Percentage of exports compared to imports in NO2 with Germany")
plt.ylabel("Percentage [%]")
plt.xticks(years)
plt.show()

Summary

## Task 4
### The weekly sum of net exports from NO2 to Germany


In [ ]:
# Calculate the net export from NO2 to DE_LU each week
physicalflows_week = physicalflows.copy()
physicalflows_week["Week"] = physicalflows_week["DateTime"].dt.isocalendar().week

physicalflows_week["NetExport"] = physicalflows_week.apply(lambda row: row["FlowValue"] if row["InMapCode"] == "DE_LU" else -row["FlowValue"], axis=1)

weekly_net_exports = physicalflows_week.resample('W', on='DateTime')['NetExport'].sum()

# Plot the weekly net exports
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(weekly_net_exports.index, weekly_net_exports.values, marker="o")
ax.axhline(0, color='red', linestyle='--', linewidth=1, label='Net Export = 0')
ax.set_xlabel('Week')
ax.set_ylabel('Net Electricity Flow (MWh)')
ax.set_title('Weekly Net Electricity Exports from NO2 to Germany via Nordlink')
ax.grid(True)
plt.savefig("figure_task4.png")


## Task 5
### Random stuff
- a
- b

In [ ]:
# Calculate the weekly price difference between germany and no2 and plot the correlation between the weekly price different and the sum of net exports that week
dayaheadprices_weekly = dayaheadprices_hourly.copy()

prices_filtered = dayaheadprices_weekly.groupby(['DateTime', 'MapCode'])['Price'].mean().reset_index()

prices_pivot = prices_filtered.pivot(index='DateTime', columns='MapCode', values='Price')

# Calculate the price difference (Germany - NO2).

prices_pivot['PriceDifference'] = prices_pivot['DE_LU'] - prices_pivot['NO2']

# Resample price difference to weekly average.

weekly_price_diff = prices_pivot['PriceDifference'].resample('W').mean()

# Combine weekly net exports and weekly average price difference.

weekly_data = pd.DataFrame({'NetExports': weekly_net_exports, 'PriceDifference': weekly_price_diff})

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(weekly_data['PriceDifference'], weekly_data['NetExports'], alpha=0.6)
# Calculate the correlation coefficient
correlation = weekly_data.corr().loc['PriceDifference', 'NetExports']

# Plot on a scatter plot
ax.set_xlabel('Weekly Average Price Difference (Germany - NO2) (EUR/MWh)')
ax.set_ylabel('Weekly Net Exports (MWh)')
ax.set_title(f'Weekly Net Exports vs. Price Difference\nCorrelation: {correlation:.2f}')
ax.grid(True)
fig.savefig("figure_task5.png")